In [ ]:
# It was removed from bottom cells of vanilla_ae_multi_class_sn

In [ ]:
#TSNE PLOTTING
from tf_lab.autopredictors.scripts.virt_scan_data import all_classes as vscan_categories
from tf_lab.nips.shape_net import shape_net_core_synth_id_to_category as syn_to_cat
from tf_lab.autopredictors.scripts.virt_scan_data import plotting_default_params
from general_tools.clustering.plt import plot_2d_embedding_in_grid_greedy_way, plot_2d_embedding_in_grid_forceful

all_categories = [syn_to_cat[s] for s in full_syn_ids]
mask_on_vcats = [c in vscan_categories for c in all_categories]
mask_on_vcats = np.array(mask_on_vcats, dtype=np.bool)
in_vcat_index = np.arange(len(mask_on_vcats))[mask_on_vcats]

full_pclouds_tmp = full_pclouds[in_vcat_index]
syn_ids_tmp = full_syn_ids[in_vcat_index]
model_unames_tmp = model_unames[in_vcat_index] 


def random_samples_from_each_class(syn_ids, samples_per_class=100):
    selected = []
    for syn_id in np.unique(syn_ids):
        in_class = np.where(syn_ids == syn_id)[0]
        effective_samples = min(len(in_class), samples_per_class)
        selected += np.ndarray.tolist(np.random.choice(in_class, effective_samples, replace=False))
    return selected

def save_images_of_pointclouds(pclouds, model_unames, syn_ids, save_image_dir):
    plt.ioff()   
    for i, pc in enumerate(pclouds):
        category_i = shape_net_core_synth_id_to_category[syn_ids[i]]
        params = plotting_default_params(category_i)        
        save_name = model_unames[i] + '.png'
        fig = Point_Cloud(points=pc).plot(show=False, show_axis=False, **params);
        fig.savefig(osp.join(save_image_dir, save_name))
        plt.close()



top_image_dir = osp.join(full_pclouds_path, 'images')
create_dir(top_image_dir)
samples_per_class = 200
rand_ind = random_samples_from_each_class(syn_ids_tmp, samples_per_class)
save_images_of_pointclouds(full_pclouds_tmp[rand_ind], model_unames_tmp[rand_ind],
                           syn_ids_tmp[rand_ind], top_image_dir)

In [ ]:
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset
from sklearn.manifold import TSNE
tsne_dataset = PointCloudDataSet(full_pclouds_tmp[rand_ind], labels=model_unames_tmp[rand_ind], init_shuffle=False)
feed, latent_codes, ids = latent_embedding_of_entire_dataset(tsne_dataset, ae, conf)
model = TSNE(n_components=2, random_state=seed, init='pca', verbose=True)
tsne_small = model.fit_transform(latent_codes)

In [ ]:
# Make a list with the file-names of all to be used images.
image_files = []
for s_id in model_unames_tmp[rand_ind]:
    save_name = s_id + '.png'
    image_files.append(osp.join(top_image_dir, save_name))

In [ ]:
plot_2d_embedding_in_grid_forceful(tsne_small, image_files, big_dim=8000, small_dim=200, save_file='tsne_complete_ae.png');